In [1]:
import pandas as pd
import geopandas
import altair as alt

# Part 1: Data Preprocessing
- Remove unnecessary columns
- Merge with countries geometry data

In [2]:
df = pd.read_csv("data.csv")
countries = geopandas.read_file("ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Alpha-2 code', 'Numeric code', 'Latitude (average)', 'Longitude (average)'])

- Make the country names corresponding in two datasets

In [3]:
df['Country'] = df['Country'].replace('United States', 'United States of America')
df['Country'] = df['Country'].replace('Hong Kong', 'Hong Kong S.A.R.')
df['Country'] = df['Country'].replace('Palestinian Territory, Occupied', 'Palestine')
df['Country'] = df['Country'].replace('Bahamas', 'The Bahamas')
df['Country'] = df['Country'].replace('Czech Republic', 'Czechia')
df['Country'] = df['Country'].replace('Russian Federation', 'Russia')
df['Country'] = df['Country'].replace('Korea, Republic of', 'South Korea')
df['Country'] = df['Country'].replace('Congo, the Democratic Republic of the', 'Democratic Republic of the Congo')
df['Country'] = df['Country'].replace('Venezuela, Bolivarian Republic of', 'Venezuela')
df['Country'] = df['Country'].replace('Iran, Islamic Republic of', 'Iran')
df['Country'] = df['Country'].replace('Syrian Arab Republic', 'Syria')
df['Country'] = df['Country'].replace('Viet Nam', 'Vietnam')
df['Country'] = df['Country'].replace('Holy See (Vatican City State)', 'Vatican')
df['Country'] = df['Country'].replace('Serbia', 'Republic of Serbia')
df['Country'] = df['Country'].replace('Tanzania, United Republic of', 'United Republic of Tanzania')

- Merge two datasets by country

In [5]:
geometry_df = pd.merge(countries[['ADMIN', 'geometry']], df, left_on="ADMIN", right_on="Country", how="right")
geometry_df = geometry_df.drop(columns=['Country', 'Alpha-3 code'])
geometry_df = geometry_df.rename(columns={'ADMIN': 'Country'})
geometry_df.head()

,Country,geometry,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],US,1.0,NaN,NaN,NaN,0.600,NaN
1,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,United States of America,"MULTIPOLYGON (((-122.75302 48.99251, -122.6532...",tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",US,NaN,tt0068473,7.7,107673.0,10.010,7.300
3,United Kingdom,"MULTIPOLYGON (((-7.24710 55.06932, -7.25674 55...",tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
4,United Kingdom,"MULTIPOLYGON (((-7.24710 55.06932, -7.25674 55...",tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']",GB,NaN,tt0061578,7.7,72662.0,20.398,7.600


- Create new column for every unique genre

In [6]:
import ast
unique_genres = set()
for index, row in geometry_df.iterrows():
    genres_list = ast.literal_eval(row['genres'])
    unique_genres.update(genres_list)

for genre in unique_genres:
    geometry_df[genre] = geometry_df['genres'].str.contains(genre)

In [7]:
columns_to_keep = [col for col in geometry_df.columns if col not in ['geometry']]
no_geometry_df = geometry_df[columns_to_keep]
no_geometry_df.head()

Country        id                                title  \
0  United States of America  ts300399  Five Came Back: The Reference Films   
1  United States of America   tm84618                          Taxi Driver   
2  United States of America  tm154986                          Deliverance   
3            United Kingdom  tm127384      Monty Python and the Holy Grail   
4            United Kingdom  tm120801                      The Dirty Dozen   

    type                                        description  release_year  \
0   SHOW  This collection includes 12 World War II-era p...          1945   
1  MOVIE  A mentally unstable Vietnam War veteran works ...          1976   
2  MOVIE  Intent on seeing the Cahulawassee River before...          1972   
3  MOVIE  King Arthur, accompanied by his squire, recrui...          1975   
4  MOVIE  12 American military prisoners in World War II...          1967   

  age_certification  runtime                                       genres  \
0             TV-MA       51                            ['documentation']   
1                 R      114                           ['drama', 'crime']   
2                 R      109  ['drama', 'action', 'thriller', 'european']   
3                PG       91              ['fantasy', 'action', 'comedy']   
4               NaN      150                            ['war', 'action']   

  production_countries  ...  animation horror  sport  crime  documentation  \
0                   US  ...      False  False  False  False           True   
1                   US  ...      False  False  False   True          False   
2                   US  ...      False  False  False  False          False   
3                   GB  ...      False  False  False  False          False   
4                   GB  ...      False  False  False  False          False   

   action  european  music  drama  comedy  
0   False     False  False  False   False  
1   False     False  False   True   False  
2    True      True  False   True   False  
3    True     False  False  False    True  
4    True     False  False  False   False  

[5 rows x 35 columns]

# Part 2: Data Visualizations

`geometry_df` is the dataframe that contains `geometry` for every country. It will be used for our data visualizations that are maps. `geometry_df` is a very big dataframe (> 4 GB), considering that it might take long to produce data visualizations that doesn't use geometry, `no_geometry_df` is the dataframe without `geometry` column. 

In [8]:
import pandas as pd

# Assuming your DataFrame is named 'df'
country_counts = no_geometry_df['production_countries'].value_counts()
top_3_countries = country_counts.nlargest(3).index.tolist()

top_3_df = no_geometry_df[no_geometry_df['production_countries'].isin(top_3_countries)]

In [9]:
top_3_df

,Country,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,...,animation,horror,sport,crime,documentation,action,european,music,drama,comedy
0,United States of America,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],US,...,False,False,False,False,True,False,False,False,False,False
1,United States of America,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",US,...,False,False,False,True,False,False,False,False,True,False
2,United States of America,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",US,...,False,False,False,False,False,True,True,False,True,False
5,United States of America,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']",US,...,False,False,False,False,False,True,False,False,False,False
8,United States of America,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'action', 'crime']",US,...,False,False,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6724,South Korea,tm831387,What Happened to Mr Cha?,MOVIE,"With the peak of his career long behind him, a...",2021,NaN,102,"['comedy', 'drama']",KR,...,False,False,False,False,False,False,False,False,True,True
6731,India,ts273317,Pitta Kathalu,SHOW,"Four different women, four journeys of love an...",2021,TV-MA,37,"['drama', 'romance']",IN,...,False,False,False,False,False,False,False,False,True,False
6733,India,tm996479,Tuesdays And Fridays,MOVIE,Two millennials get into a relationship where ...,2021,NaN,106,"['romance', 'comedy']",IN,...,False,False,False,False,False,False,False,False,False,True
6744,United States of America,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021,NaN,35,[],US,...,False,False,False,False,False,False,False,False,False,False


In [12]:
import altair as alt

# Load and preprocess your data
data = top_3_df

options = [None, 'US', 'IN', 'KR']
labels = ['All', 'US', 'IN', 'KR']

# Add a dropdown widget to select the country
input_dropdown = alt.binding_select(options=options, 
                                    labels=labels,
                                    name='Countries')

selection = alt.selection_point(fields=['production_countries'], bind=input_dropdown)

# Add a brush selection tool
brush = alt.selection_interval(encodings=['x'])


# Create a base chart
base = alt.Chart(data).mark_line().encode(
    x='release_year:N',
    y='count()',
    color='type'
).transform_filter(
    selection
).add_params(
    selection,
    brush
).properties(
    width = 800,
    height = 400
)

# Create a multi-view object with the dropdown selection filter


bars = alt.Chart(top_3_df).mark_bar().encode(
    x='count()',
    y='type:N',
    color = 'type:N'
).transform_filter(
    brush
).properties(
    width = 800
)

# Create the bar chart
genre = alt.Chart(top_3_df).mark_bar().encode(
    x='count()',
    y='genres:N',
    color = 'genres:N'
).transform_filter(
    brush
)
# Display the chart
chart = base & bars

In [14]:
import json
json_chart = chart.to_dict()

with open('release.json', 'w') as f:
    json.dump(json_chart, f)

In [15]:
box = alt.Chart(top_3_df).mark_boxplot(extent="min-max").encode(
    alt.X("type:N", title="Content Type"),
    alt.Y("runtime:Q").scale(zero=False),
    alt.Color("type:N").scale(scheme="lightgreyred"),
    alt.Tooltip(['runtime:Q', 'min:Q', 'max:Q']),
).properties(
    width=200,
    height=200
).facet(
    facet = 'production_countries:N',
    title= 'The top 3 production companies and their respective content runtimes'
)

In [16]:
import json
json_chart_box = box.to_dict()

with open('boxplot.json', 'w') as f:
    json.dump(json_chart_box, f)